In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
import numpy as np
import glob
import gc
import os
from tqdm import tqdm

## Loading Stock Data

This section reads all CSV files from the specified folder containing historical stock data for NASDAQ-listed companies. Each file corresponds to a single stock and is stored in a dictionary (`stock_data`) using the stock ticker as the key.

For each file:
- Unnecessary columns (`Date`, `Open`, `High`, `Low`, `Close`) are dropped.
- Missing values are removed.
- A new feature `Daily_Return_%` is calculated as the percentage change in adjusted closing price.

In [6]:
# Path to the folder
folder_path = '/media/mahir_uddin/Mahir/5-2/Data Analytics/Projects/Stock Trend/Assignment Dataset/stocks'

# Get list of CSV files
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Load into dictionary
stock_data = {}

for file in tqdm(csv_files):
    try:
        ticker = os.path.basename(file).replace('.csv', '')
        df = pd.read_csv(file)
        df = df.drop(columns=['Date','Open','High','Low','Close'])
        df = df.dropna()
        df['Daily_Return_%'] = df['Adj Close'].pct_change(fill_method=None).fillna(0) * 100
        stock_data[ticker] = df   
        
        del df
        gc.collect()
        
    except Exception as e:
        print(f"Error loading {file}: {e}")


100%|███████████████████████████████████████| 5884/5884 [15:07<00:00,  6.48it/s]
